<a href="https://colab.research.google.com/github/tompoles/Portfolio/blob/project_3/Customer_Segmetation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import time
import datetime as dt
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount ('/gdrive')
%cd /gdrive/MyDrive/Colab Notebooks/Datasets

Mounted at /gdrive
/gdrive/MyDrive/Colab Notebooks/Datasets


In [3]:
df = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Datasets/cust_seg.csv', encoding='latin1')

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
df.shape

(541909, 8)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [7]:
df.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
Country             0
UnitPrice           0
InvoiceDate         0
Quantity            0
StockCode           0
InvoiceNo           0
dtype: int64

In [8]:
data = pd.DataFrame(df.isnull().sum()/df.shape[0]*100)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, InvoiceNo to Country
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       8 non-null      float64
dtypes: float64(1)
memory usage: 448.0+ bytes


In [10]:
data.rename(columns={' ':'% Null Values'}, inplace=True)

In [11]:
pd.DataFrame(data = (df.isna().sum() / df.shape[0]) * 100, index = df.columns, columns = ['% Null Values'])
# Around 25% of transactions do not have a CustomerID and .268% of transactions do not have a Description of product

,% Null Values
InvoiceNo,0.000000
StockCode,0.000000
Description,0.268311
Quantity,0.000000
InvoiceDate,0.000000
UnitPrice,0.000000
CustomerID,24.926694
Country,0.000000


In [12]:
# Dropnuti zakazek bez Custumer ID
df.dropna(subset=['CustomerID'], inplace=True)

In [13]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
df.duplicated().sum()

5225

In [15]:
# Kontorla duplikatu a jejich vymazani
df.drop_duplicates(inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    401604 non-null  object 
 1   StockCode    401604 non-null  object 
 2   Description  401604 non-null  object 
 3   Quantity     401604 non-null  int64  
 4   InvoiceDate  401604 non-null  object 
 5   UnitPrice    401604 non-null  float64
 6   CustomerID   401604 non-null  float64
 7   Country      401604 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.6+ MB


In [17]:
# Odstraneni zrusenych objednavek
df = df[df['Quantity'] > 0]

In [18]:
df.shape

(392732, 8)

In [19]:
# Vraci pocet odlisnych pozorovani v objednavkach
df.InvoiceNo.nunique()

18536

In [20]:
pd.DataFrame(data=[df['InvoiceNo'].nunique(),df['StockCode'].nunique(),df['CustomerID'].nunique()],columns=['Count'],
                   index=['Number of Transactions','Number of Unique Products Bought','Number of Unique Customers'])

,Count
Number of Transactions,18536
Number of Unique Products Bought,3665
Number of Unique Customers,4339


RFM analýza
Analýza RFM (Aktuálnost, Frekvence, Monetární) je technika segmentace zákazníků, která k rozdělení zákazníků do skupin využívá chování z minulých nákupů.
RFM pomáhá rozdělit zákazníky do různých kategorií nebo klastrů, aby identifikovali zákazníky, u nichž je větší pravděpodobnost, že budou reagovat na propagační akce a také pro budoucí personalizační služby.

*   RECENCY (R): Days since last purchase
*   FREQUENCY (F): Total number of purchases
*   MONETARY VALUE (M): Total money this customer spent.

In [21]:
# Aktuálnost, Pro výpočet aktuálnosti musíme vybrat datum, od kterého vyhodnotíme, kolik dní předtím byl poslední nákup zákazníka.
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64')

In [22]:
df.InvoiceDate.sort_values(ascending=False).head()

541908   2011-12-09 12:50:00
541900   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541895   2011-12-09 12:50:00
541896   2011-12-09 12:50:00
Name: InvoiceDate, dtype: datetime64[ns]

In [23]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [24]:
now = dt.datetime(2011,12,9)
print(now)

2011-12-09 00:00:00


In [25]:
df['Date'] = df['InvoiceDate'].apply(lambda x: x.date())

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392732 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392732 non-null  object        
 1   StockCode    392732 non-null  object        
 2   Description  392732 non-null  object        
 3   Quantity     392732 non-null  int64         
 4   InvoiceDate  392732 non-null  datetime64[ns]
 5   UnitPrice    392732 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      392732 non-null  object        
 8   Date         392732 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.0+ MB


In [27]:
# Groub by Customer id k datu nakupu
recency_df = df.groupby(by='CustomerID', as_index=False)['Date'].max()
recency_df['CustomerID'] = recency_df['CustomerID'].astype('int')

In [28]:
recency_df.rename(columns={'Date':'PurchaseDate'}, inplace=True)

In [29]:
recency_df['PurchaseDate'] = recency_df['PurchaseDate'].astype('datetime64') # zmena datoveho typu u data

In [30]:
recency_df['Recency'] = recency_df['PurchaseDate'].apply(lambda x: (now - x).days) # determinovani delky mezidobi nakupu 
recency_df

,CustomerID,PurchaseDate,Recency
0,12346,2011-01-18,325
1,12347,2011-12-07,2
2,12348,2011-09-25,75
3,12349,2011-11-21,18
4,12350,2011-02-02,310
...,...,...,...
4334,18280,2011-03-07,277
4335,18281,2011-06-12,180
4336,18282,2011-12-02,7
4337,18283,2011-12-06,3


Frequency

Frekvence nám pomáhá zjistit, kolikrát od nás zákazník zakoupil. K tomu musíme zkontrolovat, kolik faktur je zaregistrováno stejným zákazníkem.

In [63]:
Freq_df = df.copy()
Freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392732 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392732 non-null  object        
 1   StockCode    392732 non-null  object        
 2   Description  392732 non-null  object        
 3   Quantity     392732 non-null  int64         
 4   InvoiceDate  392732 non-null  datetime64[ns]
 5   UnitPrice    392732 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      392732 non-null  object        
 8   Date         392732 non-null  object        
 9   TotalCost    392732 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 33.0+ MB


In [54]:
Freq_df['InvoiceNo']=Freq_df['InvoiceNo'].astype('int')

In [34]:
Freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392732 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392732 non-null  int64         
 1   StockCode    392732 non-null  object        
 2   Description  392732 non-null  object        
 3   Quantity     392732 non-null  int64         
 4   InvoiceDate  392732 non-null  datetime64[ns]
 5   UnitPrice    392732 non-null  float64       
 6   CustomerID   392732 non-null  int64         
 7   Country      392732 non-null  object        
 8   Date         392732 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 30.0+ MB


In [36]:
Freq_df.CustomerID.count()

18536

In [55]:
Freq_df.drop_duplicates(['CustomerID'], keep='first', inplace=True)

In [66]:
Freq_df['CustomerID']=Freq_df['CustomerID'].astype('int')

In [70]:
Freq_df.drop_duplicates(['InvoiceNo'], keep='first', inplace=True)

In [71]:
Frequency_df = Freq_df.groupby(by=['CustomerID'], as_index=False).agg({'InvoiceNo':'count'})

In [72]:
# Pocet zakazek od stejneho zakaznika
Frequency_df

,CustomerID,InvoiceNo
0,12346,1
1,12347,7
2,12348,4
3,12349,1
4,12350,1
...,...,...
4334,18280,1
4335,18281,1
4336,18282,2
4337,18283,16


Monetary

Peněžní atribut odpovídá na otázku: Kolik peněz zákazník v průběhu času utratil?

K tomu nejdříve vytvoříme nový sloupec celkové náklady, abychom měli celkovou cenu za fakturu.

In [41]:
df['TotalCost'] = df['Quantity'] * df['UnitPrice']

In [42]:
monetary_df = df.groupby(by='CustomerID', as_index=False).agg({'TotalCost':'sum'})

In [43]:
monetary_df.columns = ['CustomerID','Monetary']

In [44]:
monetary_df

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4334,18280.0,180.60
4335,18281.0,80.82
4336,18282.0,178.05
4337,18283.0,2045.53


Create RFM Table

In [73]:
rfm_df = recency_df.merge(Frequency_df, on='CustomerID').merge(monetary_df, on='CustomerID')

In [76]:
rfm_df.drop(columns=['PurchaseDate'], inplace=True)

In [75]:
rfm_df.rename(columns={'InvoiceNo':'Frequency'}, inplace=True)

In [77]:
rfm_df.set_index('CustomerID', inplace=True)

In [78]:
rfm_df

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,18,1,1757.55
12350,310,1,334.40
...,...,...,...
18280,277,1,180.60
18281,180,1,80.82
18282,7,2,178.05


In [79]:
# rfm_df.to_csv('/gdrive/MyDrive/Colab Notebooks/Datasets/rfm.csv')

Segmenty zákazníků s modelem RFM

Než přejdeme k zákaznickým segmentům, podívejme se na aplikaci Paretova principu - běžně označovaného jako pravidlo 80–20 v naší datové sadě jeho aplikací na naše proměnné RFM.

Paretovo pravidlo říká, že 80% výsledků pochází z 20% příčin.

Podobně 20% zákazníků přispívá k 80% vašich celkových příjmů. Ověřme si to, protože to nám pomůže zjistit, na které zákazníky se při marketingu nových produktů zaměřit.

In [81]:
pareto_df = rfm_df['Monetary'].sum() * 0.8
print("The 80% of total revenue is: ",round(pareto_df,2))

The 80% of total revenue is:  7109767.12


In [82]:
# Ohodnoceni zakazniku adekvatne k obratu
customers_ranked = rfm_df
customers_ranked['Rank'] = customers_ranked['Monetary'].rank(ascending=False)
customers_ranked.head()

,Recency,Frequency,Monetary,Rank
CustomerID,,,,
12346,325,1,77183.60,10.0
12347,2,7,4310.00,335.0
12348,75,4,1797.24,1004.0
12349,18,1,1757.55,1027.0
12350,310,1,334.40,3097.0


In [87]:
customers_ranked.sort_values(by='Rank', ascending=True, inplace=True)

In [94]:
customers_ranked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339 entries, 14646 to 13256
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recency    4339 non-null   int64  
 1   Frequency  4339 non-null   int64  
 2   Monetary   4339 non-null   float64
 3   Rank       4339 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 169.5 KB


In [92]:
top_20cust_df = (4339*20)/100 
top_20cust_df

867.8

In [98]:
top20_revenue_df = customers_ranked[customers_ranked['Rank'] <= 867.8]['Monetary'].sum()
top20_revenue_df, pareto_df

(6635245.310999999, 7109767.1152)

Pozorujeme, že výnosy generované 20% nejlepších zákazníků jsou o něco méně než 80% celkových výnosů. Tato dvě čísla však nejsou vždy úplně stejná, ale v našem případě jsou si velmi blízká. Dalo nám to tedy dobrý signál o tom, že Paretovo pravidlo platí.

In [101]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary,Rank
0.25,17.0,1.0,306.455,1085.5
0.50,50.0,2.0,668.560,2170.0
0.75,141.5,5.0,1660.315,3254.5


In [102]:
quantiles.to_dict()

{'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'Monetary': {0.25: 306.45500000000004,
  0.5: 668.5600000000002,
  0.75: 1660.315000000001},
 'Rank': {0.25: 1085.5, 0.5: 2170.0, 0.75: 3254.5},
 'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 141.5}}

Vytvoření segmentační tabulky RFM

Vytvoříme dvě podmínky segmentace, jednu pro aktuálnost a druhou pro fequency a peněžní. Je to proto, že vysoká aktuálnost je špatná, zatímco vysoká frekvence a peněžní hodnota jsou dobré.

In [104]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def Rscore(x, p, d):
  if x <= d [p][0.25]:
    return 4 
  elif x <= d[p][0.50]:
    return 3
  elif x <= d[p][0.75]: 
    return 2
  else:
    return 1

In [105]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [107]:
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(Rscore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [108]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile
CustomerID,,,,,,,
14646,1,74,280206.02,1.0,4,4,4
18102,0,60,259657.30,2.0,4,4,4
17450,8,46,194390.79,3.0,4,4,4
16446,0,2,168472.50,4.0,4,2,4
14911,1,201,143711.17,5.0,4,4,4


Nyní, když máme skóre každého zákazníka, můžeme představovat naši segmentaci zákazníků.
Nejprve musíme kombinovat skóre (R_Quartile, F_Quartile, M_Quartile) dohromady.

In [116]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str)+rfm_segmentation.F_Quartile.map(str)+rfm_segmentation.M_Quartile.map(str)
rfm_segmentation

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,,
14646,1,74,280206.02,1.0,4,4,4,444
18102,0,60,259657.30,2.0,4,4,4,444
17450,8,46,194390.79,3.0,4,4,4,444
16446,0,2,168472.50,4.0,4,2,4,424
14911,1,201,143711.17,5.0,4,4,4,444
...,...,...,...,...,...,...,...,...
17956,249,1,12.75,4335.0,1,1,1,111
16454,44,2,6.90,4336.0,3,2,1,321
14792,63,1,6.20,4337.0,2,1,1,211


In [118]:
cust_seg_df = rfm_segmentation[rfm_segmentation['RFMScore']== '444'].sort_values(by='Monetary', ascending=False)
cust_seg_df.head(10)

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,,
14646,1,74,280206.02,1.0,4,4,4,444
18102,0,60,259657.30,2.0,4,4,4,444
17450,8,46,194390.79,3.0,4,4,4,444
14911,1,201,143711.17,5.0,4,4,4,444
14156,9,55,117210.08,7.0,4,4,4,444
17511,2,31,91062.38,8.0,4,4,4,444
16684,4,28,66653.56,11.0,4,4,4,444
14096,4,17,65164.79,12.0,4,4,4,444
13694,3,50,65039.62,13.0,4,4,4,444


In [119]:
# cust_seg_df.to_csv('/gdrive/MyDrive/Colab Notebooks/Datasets/cust_seg_RFM.csv')